# PostgreSQL Vector Database Query Notebook

This notebook provides an interface to query a PostgreSQL vector database.

In [26]:
# Set your PostgreSQL connection parameters here
DB_HOST = 'pgvector.llama-stack-rag.svc.cluster.local'
DB_PORT = '5432'
DB_NAME = 'rag_blueprint'
DB_USER = 'postgres'
DB_PASSWORD = 'rag_password'
TABLE_NAME = 'vector_store_demo_rag_vector_db_v1_0'  # Specific table to query, or None to query all tables
ROW_LIMIT = 50  # Limit number of rows to display
SHOW_VECTORS = False  # Set to True to include vector columns in output

In [27]:
import sys
import os
import psycopg2
from tabulate import tabulate
import pandas as pd

In [28]:
def connect_to_db(host, port, dbname, user, password):
    """Connect to PostgreSQL database"""
    try:
        conn = psycopg2.connect(
            host=host,
            port=port,
            dbname=dbname,
            user=user,
            password=password
        )
        return conn
    except Exception as e:
        print(f"Error connecting to database: {e}")
        return None

def list_tables(conn):
    """List all tables in the database"""
    cursor = conn.cursor()
    cursor.execute("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public'
    """)
    tables = cursor.fetchall()
    cursor.close()
    return [table[0] for table in tables]

def query_table(conn, table_name, limit=50, exclude_vector_cols=True):
    """Query a table and return results as a DataFrame"""
    cursor = conn.cursor()
    
    # Get column names first
    cursor.execute(f"SELECT * FROM {table_name} LIMIT 0")
    column_names = [desc[0] for desc in cursor.description]
    
    # If excluding vector columns, identify them
    if exclude_vector_cols:
        vector_cols = []
        for col in column_names:
            try:
                cursor.execute(f"SELECT pg_typeof({col}) FROM {table_name} LIMIT 1")
                col_type = cursor.fetchone()[0]
                if 'vector' in col_type.lower():
                    vector_cols.append(col)
            except:
                pass
        
        # Create a query that selects all columns except vector columns
        if vector_cols:
            select_cols = [col for col in column_names if col not in vector_cols]
            query = f"SELECT {', '.join(select_cols)} FROM {table_name} LIMIT {limit}"
        else:
            query = f"SELECT * FROM {table_name} LIMIT {limit}"
    else:
        query = f"SELECT * FROM {table_name} LIMIT {limit}"
    
    cursor.execute(query)
    rows = cursor.fetchall()
    
    # Get the actual column names from the cursor description
    columns = [desc[0] for desc in cursor.description]
    
    cursor.close()
    return pd.DataFrame(rows, columns=columns)

def describe_table(conn, table_name):
    """Get the structure of a table"""
    cursor = conn.cursor()
    cursor.execute(f"""
        SELECT column_name, data_type, character_maximum_length 
        FROM information_schema.columns 
        WHERE table_name = '{table_name}'
    """)
    columns = cursor.fetchall()
    cursor.close()
    return pd.DataFrame(columns, columns=['Column', 'Type', 'Max Length'])

In [29]:
# Connect to the database
conn = connect_to_db(DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD)
if not conn:
    raise Exception("Failed to connect to database")

In [30]:
# List all tables in the database
tables = list_tables(conn)
print(f"Found {len(tables)} tables: {', '.join(tables)}\n")

Found 2 tables: metadata_store, vector_store_demo_rag_vector_db_v1_0



In [31]:
# Query the specified table or all tables
if TABLE_NAME:
    if TABLE_NAME not in tables:
        print(f"Table '{TABLE_NAME}' not found in database.")
    else:
        print(f"Table structure for '{TABLE_NAME}':")
        structure = describe_table(conn, TABLE_NAME)
        display(structure)
        print()
        
        print(f"Sample data from '{TABLE_NAME}' (limit: {ROW_LIMIT}):")
        df = query_table(conn, TABLE_NAME, ROW_LIMIT, not SHOW_VECTORS)
        display(df)
else:
    for table in tables:
        print(f"Table structure for '{table}':")
        structure = describe_table(conn, table)
        display(structure)
        print()
        
        print(f"Sample data from '{table}' (limit: {ROW_LIMIT}):")
        df = query_table(conn, table, ROW_LIMIT, not SHOW_VECTORS)
        display(df)
        print("\n" + "-" * 80 + "\n")

Table structure for 'vector_store_demo_rag_vector_db_v1_0':


,Column,Type,Max Length
0,document,jsonb,None
1,embedding,USER-DEFINED,None
2,id,text,None



Sample data from 'vector_store_demo_rag_vector_db_v1_0' (limit: 50):


,id,document
0,doc-1:chunk-0,{'content': 'Version 1.0 Christoph Auer Maksym...
1,doc-2:chunk-1,{'content': 'This technical report introduces ...
2,doc-3:chunk-2,{'content': 'Converting PDF documents back int...
3,doc-4:chunk-3,"{'content': 'To use Docling, you can simply in..."
4,doc-5:chunk-4,{'content': 'Docling implements a linear pipel...
5,doc-6:chunk-5,{'content': 'Two basic requirements to process...
6,doc-7:chunk-6,"{'content': 'As part of Docling, we initially ..."
7,doc-8:chunk-7,{'content': 'Our layout analysis model is an o...
8,doc-9:chunk-8,"{'content': 'The TableFormer model [12], first..."
9,doc-10:chunk-9,{'content': 'Docling provides optional support...


In [ ]:
!pip install psycopg2 psycopg-binary

In [32]:
# Close the database connection
conn.close()

## Custom Queries

You can execute custom SQL queries against the database below.

In [ ]:
def execute_custom_query(query, limit=100):
    """Execute a custom SQL query and return results as a DataFrame"""
    conn = connect_to_db(DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD)
    if not conn:
        return None
    
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        
        if query.strip().upper().startswith('SELECT'):
            rows = cursor.fetchmany(limit)
            columns = [desc[0] for desc in cursor.description]
            return pd.DataFrame(rows, columns=columns)
        else:
            conn.commit()
            return pd.DataFrame([['Query executed successfully']], columns=['Result'])
    except Exception as e:
        return pd.DataFrame([[f"Error: {str(e)}"]], columns=['Result'])
    finally:
        conn.close()

In [ ]:
# Example custom query
custom_query = f"SELECT * FROM {TABLE_NAME} LIMIT 10"
result = execute_custom_query(custom_query)
display(result)